In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import pickle as pkl

from sklearn.metrics import precision_recall_fscore_support

import string
import logging

import glob
import tqdm
import pickle

from nlstruct.utils import torch_clone
from nlstruct.utils import torch_global as tg
from nlstruct.dataloaders import load_from_brat


sys.path.insert(0,'/export/home/cse200093/deep_mlg_normalization/deep_multilingual_normalization')
sys.path.insert(0,'/export/home/cse200093/deep_mlg_normalization')

from deep_multilingual_normalization.preprocess import preprocess, load_quaero
from deep_multilingual_normalization.train import train_step1, train_step2, clear
from deep_multilingual_normalization.eval import predict

from notebook_utils import *

print('done')

In [ ]:
# load cui2sty
open_file = open('/export/home/cse200093/Expe_Translation/english_Norm/umls_en_fr_cui2sty.pkl', "rb")
umls_cui2sty = pickle.load(open_file)
open_file.close()

In [ ]:
# map sty group to semantic group
sty2type = {
    'Activity': 'ACTI',
    'Behavior': 'ACTI',
    'Daily or Recreational Activity': 'ACTI',
    'Event': 'ACTI',
    'Governmental or Regulatory Activity': 'ACTI',
    'Individual Behavior': 'ACTI',
    'Machine Activity': 'ACTI',
    'Occupational Activity': 'ACTI',
    'Social Behavior': 'ACTI',
    'Anatomical Structure': 'ANAT',
    'Body Location or Region': 'ANAT',
    'Body Part, Organ, or Organ Component': 'ANAT',
    'Body Space or Junction': 'ANAT',
    'Body Substance': 'ANAT',
    'Body System': 'ANAT',
    'Cell': 'ANAT',
    'Cell Component': 'ANAT',
    'Embryonic Structure': 'ANAT',
    'Fully Formed Anatomical Structure': 'ANAT',
    'Tissue': 'ANAT',
    'Amino Acid, Peptide, or Protein': 'CHEM',
    'Antibiotic': 'CHEM',
    'Biologically Active Substance': 'CHEM',
    'Biomedical or Dental Material': 'CHEM',
    'Chemical': 'CHEM',
    'Chemical Viewed Functionally': 'CHEM',
    'Chemical Viewed Structurally': 'CHEM',
    'Clinical Drug': 'CHEM',
    'Element, Ion, or Isotope': 'CHEM',
    'Enzyme': 'CHEM',
    'Hazardous or Poisonous Substance': 'CHEM',
    'Hormone': 'CHEM',
    'Immunologic Factor': 'CHEM',
    'Indicator, Reagent, or Diagnostic Aid': 'CHEM',
    'Inorganic Chemical': 'CHEM',
    'Nucleic Acid, Nucleoside, or Nucleotide': 'CHEM',
    'Organic Chemical': 'CHEM',
    'Pharmacologic Substance': 'CHEM',
    'Receptor': 'CHEM',
    'Vitamin': 'CHEM',
    'Classification': 'CONC',
    'Conceptual Entity': 'CONC',
    'Functional Concept': 'CONC',
    'Group Attribute': 'CONC',
    'Idea or Concept': 'CONC',
    'Intellectual Product': 'CONC',
    'Language': 'CONC',
    'Qualitative Concept': 'CONC',
    'Quantitative Concept': 'CONC',
    'Regulation or Law': 'CONC',
    'Spatial Concept': 'CONC',
    'Temporal Concept': 'CONC',
    'Drug Delivery Device': 'DEVI',
    'Medical Device': 'DEVI',
    'Research Device': 'DEVI',
    'Acquired Abnormality': 'DISO',
    'Anatomical Abnormality': 'DISO',
    'Cell or Molecular Dysfunction': 'DISO',
    'Congenital Abnormality': 'DISO',
    'Disease or Syndrome': 'DISO',
    'Experimental Model of Disease': 'DISO',
    'Finding': 'DISO',
    'Injury or Poisoning': 'DISO',
    'Mental or Behavioral Dysfunction': 'DISO',
    'Neoplastic Process': 'DISO',
    'Pathologic Function': 'DISO',
    'Sign or Symptom': 'DISO',
    'Amino Acid Sequence': 'GENE',
    'Carbohydrate Sequence': 'GENE',
    'Gene or Genome': 'GENE',
    'Molecular Sequence': 'GENE',
    'Nucleotide Sequence': 'GENE',
    'Geographic Area': 'GEOG',
    'Age Group': 'LIVB',
    'Amphibian': 'LIVB',
    'Animal': 'LIVB',
    'Archaeon': 'LIVB',
    'Bacterium': 'LIVB',
    'Bird': 'LIVB',
    'Eukaryote': 'LIVB',
    'Family Group': 'LIVB',
    'Fish': 'LIVB',
    'Fungus': 'LIVB',
    'Group': 'LIVB',
    'Human': 'LIVB',
    'Mammal': 'LIVB',
    'Organism': 'LIVB',
    'Patient or Disabled Group': 'LIVB',
    'Plant': 'LIVB',
    'Population Group': 'LIVB',
    'Professional or Occupational Group': 'LIVB',
    'Reptile': 'LIVB',
    'Vertebrate': 'LIVB',
    'Virus': 'LIVB',
    'Entity': 'OBJC',
    'Food': 'OBJC',
    'Manufactured Object': 'OBJC',
    'Physical Object': 'OBJC',
    'Substance': 'OBJC',
    'Biomedical Occupation or Discipline': 'OCCU',
    'Occupation or Discipline': 'OCCU',
    'Health Care Related Organization': 'ORGA',
    'Organization': 'ORGA',
    'Professional Society': 'ORGA',
    'Self-help or Relief Organization': 'ORGA',
    'Biologic Function': 'PHEN',
    'Environmental Effect of Humans': 'PHEN',
    'Human-caused Phenomenon or Process': 'PHEN',
    'Laboratory or Test Result': 'PHEN',
    'Natural Phenomenon or Process': 'PHEN',
    'Phenomenon or Process': 'PHEN',
    'Cell Function': 'PHYS',
    'Clinical Attribute': 'PHYS',
    'Genetic Function': 'PHYS',
    'Mental Process': 'PHYS',
    'Molecular Function': 'PHYS',
    'Organism Attribute': 'PHYS',
    'Organism Function': 'PHYS',
    'Organ or Tissue Function': 'PHYS',
    'Physiologic Function': 'PHYS',
    'Diagnostic Procedure': 'PROC',
    'Educational Activity': 'PROC',
    'Health Care Activity': 'PROC',
    'Laboratory Procedure': 'PROC',
    'Molecular Biology Research Technique': 'PROC',
    'Research Activity': 'PROC',
    'Therapeutic or Preventive Procedure': 'PROC'
}

In [ ]:
# path to predict data and gold standard
# the NER phase has been done in nlstruct_predict.ipynb and results saved to PREDICT_DATA_PATH
PREDICT_DATA_PATH = '/export/home/cse200093/Expe_Translation/nlstruct_EMEA_restrict_types'
STAND_DATA_PATH = '/export/home/cse200093/brat_data/QUAERO_FrenchMed/corpus/test/EMEA'

In [ ]:
# load the normalization model
model = torch.load('/export/home/cse200093/deep_mlg_normalization/models/UMLS2021AB_without_quaero.pt')
model.eval()

In [ ]:
bert_name = "/export/home/cse200093/deep_mlg_normalization/bert-base-multilingual-uncased"
VOCAB_PATH = '/export/home/cse200093/deep_mlg_normalization'

import os

with open(os.path.join(VOCAB_PATH,'vocab1.pkl'), 'rb') as f:
    vocabularies1 = pkl.load(f)

with open(os.path.join(VOCAB_PATH,'vocab2021AB_v2.pkl'), 'rb') as f:
    vocabularies2 = pkl.load(f)

In [ ]:
dataset = load_from_brat(PREDICT_DATA_PATH)

dataset['mentions']['mention_id'] = dataset['mentions']['doc_id'] +'.'+ dataset['mentions']['mention_id'].astype(str)

batcher, vocs, mention_ids = preprocess_train(
    dataset,
    vocabularies=vocabularies2,
    bert_name=bert_name,
)

batch_size = len(batcher)
with_tqdm = True

tg.set_device('cuda:6') #('cuda:0')
device = tg.device

# topk: return first k candidates
nb = 5
pred_batcher = predict(batcher, model, batch_size=64, return_loss=False, topk=nb)

print('pred_batcher', pred_batcher)
print('batcher', batcher)

In [ ]:
voc_tokens = vocs['token']
voc_labels = vocs['label']
def replace_fn(s):
    return s.replace(' ##', '').replace('[SEP]', '').replace('[PAD]', '').replace('[CLS]', '').strip()
merged_batcher = batcher['mention',['mention_id','token']].merge(pred_batcher)
final_tokens = [replace_fn(' '.join([voc_tokens[i] for i in b])) for b in merged_batcher['mention']['token'].toarray()]
final_labels = [voc_labels[b] for b in merged_batcher['mention']['label']]

final_couples = [(t, l) for t,l in zip(final_tokens, final_labels)]

# index of elements in the list is just the corresponding mention_id
final_couples

In [ ]:
# Add type information: output the first candidate with right type
def choose(l, stand):
    i = 0
    while l[i]!=stand and i<nb-1:
        i+=1
    if i == nb-1:
        return 0
    else:
        return i
    
def type_info(final_couples):
    gold_types = list(dataset['mentions']['label'])
    new_final_couples = []
    nb_unknown = 0 # number of unknown cuis
    unknown_cuis = []
    for i in tqdm.tqdm(range(len(final_couples))):
        candidates = final_couples[i][1]
        candidate_types = [sty2type[umls_cui2sty[candidate]] if candidate in umls_cui2sty.keys() else 'unknown_cui' for candidate in candidates]
        nb_unknown+=len([x for x in candidate_types if x=='unknown_cui'])
        unknown_cuis+=[candidates[i] for i in range(nb) if candidate_types[i]=='unknown_cui']
        gold_type = gold_types[i]
        res = choose(candidate_types, gold_type)
        
        new_final_couples.append((final_couples[i][0],candidates[res]))
        
    print(nb_unknown)
    return (new_final_couples,unknown_cuis)

In [ ]:
# add type info
final_couples,unknown_cuis = type_info(final_couples)
final_couples

In [ ]:
df_mentions = dataset['mentions'].copy()
df_mentions['cui'] = [x[1] for x in final_couples]
df_mentions.set_index('mention_id',inplace=True)
df_mentions

In [ ]:
os.chdir(STAND_DATA_PATH)
my_files = glob.glob('*.ann')
len(my_files)

In [ ]:
phrases = []
cuis = []
types = []
dic = {} # necessary when len(phrases) != len(cuis)
for file in tqdm.tqdm(my_files):
    phrase = None
    code = None
    type_name = None
    f = open(STAND_DATA_PATH+'/'+file, "r")
    doc = file
    for line in f:
        line = line.rstrip('\n')
        if line.startswith('T'):
            type_name = line.split('\t')[1].split(' ')[0]
            phrase = line.split('\t')[2]
            code = line.split('\t')[0] # necessary when len(phrases) != len(cuis)
            dic[file+'.'+phrase+'.'+code] = 'no cui' # necessary when len(phrases) != len(cuis)
            #phrases.append(phrase)
            #types.append(type_name)
        elif line.startswith('#'):
            cui = line.split('\t')[2]
            dic[file+'.'+phrase+'.'+code] = (cui,type_name) # necessary when len(phrases) != len(cuis)
            #cui = cui.split(',')[0][1:].rstrip('\"') # necessary for Mantra
            #cuis.append(cui)

In [ ]:
# necessary when len(phrases) != len(cuis)
print(len(dic))
no_cuis = []
docs = []
# phrases_eng = []
for key in tqdm.tqdm(dic):
    if dic[key] != 'no cui':
        phrases.append(key.split('.')[2])
        # necessary with translation
        # phrases_eng.append(translator.translate(key.split('.')[2]).text)
        cuis.append(dic[key][0])
        types.append(dic[key][1])
        docs.append(key.split('.')[0])
    else:
        #print(key)
        no_cuis.append(key)
        
print(len(cuis),len(phrases),len(types))

In [ ]:
# gold standard
df_stand = pd.DataFrame({'doc_id':docs, 'text':phrases, 'cui':cuis, 'types':types})
df_stand = df_stand[(df_stand['types']=='PROC')|(df_stand['types']=='DEVI')|(df_stand['types']=='DISO')|(df_stand['types']=='CHEM')]
df_stand

In [ ]:
def get_dic(df):
    dic = {}
    docs = list(df['doc_id'])
    for doc in docs:
        cuis = list(df[df['doc_id']==doc]['cui'])
        dic[doc] = cuis
    return dic

In [ ]:
def convert_list_to_dic(dic):
    for file_name in tqdm.tqdm(dic):
        key_list = list(set(dic[file_name]))
        value_list = [dic[file_name].count(x) for x in key_list]
        res = dict(zip(key_list, value_list))
        dic[file_name] = res

In [ ]:
dic_stand = get_dic(df_stand)
dic_res = get_dic(df_mentions)

convert_list_to_dic(dic_stand)
convert_list_to_dic(dic_res)

In [ ]:
df_compare = pd.DataFrame(columns = ['file_name','correct_ents','built_model_ents'])

for file in tqdm.tqdm(dic_stand):
    df_compare = df_compare.append({'file_name':file, 'correct_ents':dic_stand[file], 
                       'built_model_ents':dic_res[file]},
                     ignore_index=True)
    
df_compare

In [ ]:
df_compare['built_TP'] = 0
df_compare['built_FP'] = 0
df_compare['built_FN'] = 0

df_compare['built_precision'] = 0
df_compare['built_recall'] = 0
df_compare['built_f1-score'] = 0

# calculate TP,FP,FN for each model
for index, row in tqdm.tqdm(df_compare.iterrows()):
    for cui in row['built_model_ents']:
        if cui in row['correct_ents']:
            df_compare.loc[index,'built_TP'] += min(row['correct_ents'][cui], row['built_model_ents'][cui])
        else:
            df_compare.loc[index,'built_FP'] += row['built_model_ents'][cui]
    for cui in row['correct_ents']:
        if cui not in row['built_model_ents']:
            df_compare.loc[index,'built_FN'] += row['correct_ents'][cui]
            
            
# calculate precision and recall for each model
for index, row in tqdm.tqdm(df_compare.iterrows()):
    if row['built_TP'] != 0:
        df_compare.loc[index,'built_precision'] = row['built_TP']/(row['built_TP']+row['built_FP'])

    if row['built_TP'] != 0:
        df_compare.loc[index,'built_recall'] = row['built_TP']/(row['built_TP']+row['built_FN'])

# calculate f1-score for each model
for index, row in tqdm.tqdm(df_compare.iterrows()):
    if row['built_TP'] != 0:
        df_compare.loc[index,'built_f1-score'] = (2*row['built_precision']*row['built_recall'])/(row['built_precision']+row['built_recall'])
            

df_compare

In [ ]:
# calculate mean precision, mean recall, mean f1-score

built_avg_precision = np.mean(df_compare['built_precision'])
built_avg_recall = np.mean(df_compare['built_recall'])
built_avg_f1_score = np.mean(df_compare['built_f1-score'])

print(f'Average precision of the built model: {built_avg_precision}')
print(f'Average recall of the built model: {built_avg_recall}')
print(f'Average f1-score of the built model: {built_avg_f1_score}')

In [ ]:
# save df_compare for boostrap
df_compare.to_pickle('/export/home/cse200093/Expe_Translation/Axes_script/EMEA_res.pkl')